In [2]:
from sklearn import tree
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [130]:
#список пользователей по leader-id  и email из ple
users=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/ple_user.csv', \
                              sep=';',encoding='utf-8')

#список zoom_id  мероприятий и их типов
events=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/zoom_event.csv', \
                              sep=';',encoding='utf-8')
#иногда после номера мероприятия встречается пароль - выпилим его
events.meeting_id=events.meeting_id.str.extract(r'\D*(\d*)\D*').astype('int64')

#список всех зум-сессий. В sql запросе групировка для вывода: lmp.user_name, lmp.meeting_id, lmp.email
sessions=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/zoom_sesion.csv', \
                              sep=';',encoding='utf-8')
sessions.meeting_id=sessions.meeting_id.astype('int64')


#Статистика вовлеченности за день
stat=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/engagement_by_day_v15_13_11_2020.csv', \
                              sep=';',encoding='utf-8')

In [131]:
# Сшиваем по полю meeting_id
sessions_merged = pd.merge(
    sessions,
    events,
    on=['meeting_id'],
    how='left' ).fillna(0)

# Сшиваем  c users по полю email
sessions_merged = pd.merge(
    sessions_merged,
    users,
    on=['email'],
    how='left' ).fillna(0)

sessions_merged.leaderID=sessions_merged.leaderID.astype('int')
sessions_merged['zoom_name_leaderID']= sessions_merged.user_name.str.extract(r'\D*(\d*)\D*')
sessions_merged['zoom_name_leaderID']= sessions_merged['zoom_name_leaderID'].str.lstrip('0')
sessions_merged['zoom_name_leaderID']= pd.to_numeric(sessions_merged['zoom_name_leaderID'],downcast='integer'). \
                                        fillna(0).astype('int')

sessions_merged=sessions_merged.rename(columns={ 'leaderID': 'email_leaderID'})

#Логика текущая такая: для каждого сеанса
#1. если есть в имени номер - используем его
#2. если в имени номера нет - берем leader_id
#Таким образом если с разными email но одним именем - то leaderID будет из имени браться и только при его отсутствии - из email

condlist = [sessions_merged.zoom_name_leaderID==0, sessions_merged.zoom_name_leaderID>0]
choicelist = [sessions_merged.email_leaderID, sessions_merged.zoom_name_leaderID]
sessions_merged['leaderID']= np.select(condlist,choicelist)

sessions_merged.head(3)

,user_name,meeting_id,email,time_on,День,event_type,profile,email_leaderID,zoom_name_leaderID,leaderID
0,᠌ ᠌᠌ ᠌ ᠌᠌ ᠌ ᠌ ᠌ ᠌ ᠌ ᠌᠌ ᠌ ᠌᠌ ᠌ ᠌ ᠌ ᠌,92998423778,bravo0987@bk.ru,146,07.11.2020,Акселерационная программа,devnull1578401@uni2035,1578401,0,1578401
1,᠌ ᠌᠌ ᠌ ᠌᠌ ᠌ ᠌ ᠌ ᠌ ᠌ ᠌᠌ ᠌ ᠌᠌ ᠌ ᠌ ᠌ ᠌,92998423778,bravo0987@bk.ru,146,07.11.2020,Акселерационная программа,devnull1578401@uni2035,1578401,0,1578401
2,???????? Alexei Sorokin,95230006304,plannerix@gmail.com,2986,07.11.2020,Мастер-класс,0,0,0,0


In [132]:
#Теперь делаем преобразование
#1. Отбираем тех, кто присутствовал больше 10 минут -  time_on>600
#2. Группируем по: leaderID, event_type, День
#Считаем количество мероприятий каждого типа
#

session_by_type = sessions_merged[sessions_merged.time_on>600].\
    groupby(['leaderID','День','event_type']). \
    agg(zoom_count=('meeting_id', 'count')).fillna(0)
session_by_type=session_by_type.reset_index()

#Теперь все типы Зумов сделаем колонками
session_by_day = pd.pivot_table(session_by_type,index=['leaderID','День'],columns='event_type', values='zoom_count'). \
                reset_index().fillna(0)
#Удалим не типированные Зумы
session_by_day=session_by_day.drop(columns=[0])

session_by_day.tail(3)

event_type,leaderID,День,Акселерационная программа,Визионерская лекция,Лаборатория,Лекция,Мастер-класс,Общение с экспертами,Повышение эффективности
7937,15872789,07.11.2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7938,16006847,09.11.2020,2.0,0.0,0.0,0.0,0.0,0.0,0.0
7939,322238453,07.11.2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [135]:
stat_merged.columns

Index(['leaderID', 'День', 'Загружена в PT презентация по проекту',
       'Проект в ПТ размечен', 'Участником создана задача в Командном профиле',
       'Выполнена задача в ком.профиле',
       'Верифицировано решение в ком.профиле',
       'Создание Темы изучения в Ком.профиле',
       'Закрыта Тема изучения в Ком.профиле', 'Запись на Акслератор',
       'Запись на встречу с экспертом', 'Запись на Лабораторию',
       'Запись на МК', 'Запись на повышение эффективности',
       'Запись на Питч-сессию', 'Отредактирована цель',
       'Оценено продвижение к цели в ЛК',
       'Оставлен Фидбек на мероприятие (хотя бы один ответ)',
       'Создан Продукт в Edumap', 'Поставлен лайк проекту в ПТ',
       'Поставлен лайк продукту (карточке) в Edumap', 'Создано вакансий',
       'Откликов на вакансии', 'untiID', 'Акселерационная программа',
       'Визионерская лекция', 'Лаборатория', 'Лекция', 'Мастер-класс',
       'Общение с экспертами', 'Повышение эффективности'],
      dtype='object')

In [137]:
# Сшиваем статистику с Зумами. Сшивка по полям leaderID и День
stat_merged = pd.merge(
    stat,
    session_by_day,
    on=['leaderID','День'],
    how='left' ).fillna(0)

#Переименуем в понятные всем имена - чтобы записи на мероприятия с Зумами не путались
stat_merged = stat_merged.rename(columns={ 'Акселерационная программа': 'Zoom по Акселерационной программе',
                         'Визионерская лекция':'Zoom по Визионерским лекциям',
                         'Лаборатория':'Zoom по Лабораториям',
                         'Лекция':'Zoom по Лекциям',
                         'Мастер-класс':'Zoom по Мастер-классам',
                         'Общение с экспертами':'Zoom по Общению с экспертами',
                         'Повышение эффективности': 'Zoom по Повышению эффективности'
                        })

stat_merged.head(10)

,leaderID,День,Загружена в PT презентация по проекту,Проект в ПТ размечен,Участником создана задача в Командном профиле,Выполнена задача в ком.профиле,Верифицировано решение в ком.профиле,Создание Темы изучения в Ком.профиле,Закрыта Тема изучения в Ком.профиле,Запись на Акслератор,...,Создано вакансий,Откликов на вакансии,untiID,Zoom по Акселерационной программе,Zoom по Визионерским лекциям,Zoom по Лабораториям,Zoom по Лекциям,Zoom по Мастер-классам,Zoom по Общению с экспертами,Zoom по Повышению эффективности
0,69,07.11.2020,1,1,0,0,0,0,0,2,...,2,0,14960,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,69,08.11.2020,0,0,0,0,0,0,0,2,...,0,0,14960,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,69,09.11.2020,0,0,0,0,0,0,0,2,...,0,0,14960,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,69,10.11.2020,0,0,0,0,0,0,0,2,...,0,0,14960,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,69,11.11.2020,0,0,0,0,0,0,0,2,...,0,0,14960,3.0,0.0,1.0,0.0,1.0,0.0,0.0
5,69,12.11.2020,0,0,0,0,0,0,0,2,...,0,0,14960,1.0,0.0,1.0,0.0,0.0,0.0,0.0
6,69,13.11.2020,0,0,0,0,0,0,0,2,...,0,0,14960,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,87,07.11.2020,1,1,0,0,0,0,0,2,...,1,0,25474,1.0,0.0,2.0,0.0,1.0,0.0,1.0
8,87,08.11.2020,0,0,0,0,0,0,0,2,...,0,0,25474,2.0,0.0,1.0,0.0,1.0,2.0,1.0
9,87,09.11.2020,0,0,0,0,0,0,0,2,...,0,0,25474,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [138]:
#Запишем  в файл 
stat_merged.to_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/engagement_by_day_v15_13_11_2020_fin.csv', \
                   encoding='ANSI')  